TODO: Figure out ravel() 1d array problem

In [1]:
import pandas as pd
import numpy as np
import os
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from __future__ import division
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
labels = pd.read_csv('stocknews//labels.csv', header=None)

http://stackoverflow.com/questions/28382735/python-pandas-does-not-read-the-first-row-of-csv-file

`pd.read_csv` was cutting off the first row of labels

In [3]:
# Confirm size of labels to make sure data loaded correctly
labels.shape

(1989, 1)

In [4]:
str_tokens_sw = pd.read_csv('stocknews//tokens_str_all.csv', dtype=str, keep_default_na=False)

In [5]:
str_tokens_sw.head()

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,georgia downs two russian warplanes as countri...,breaking musharraf to be impeached,russia today columns of troops roll into south...,russian tanks are moving towards the capital o...,afghan children raped with impunity official s...,150 russian tanks have entered south ossetia w...,breaking georgia invades south ossetia russia ...,the enemy combatent trials are nothing but sha...,georgian troops retreat from osettain capital ...,did the prep georgia for war with russia,...,georgia invades south ossetia if russia gets i...,al qaeda faces islamist backlash,condoleezza rice the us would not act to preve...,this is busy day the european union has approv...,georgia will withdraw 000 soldiers from iraq t...,why the pentagon thinks attacking iran is bad ...,caucasus in crisis georgia invades south ossetia,indian shoe manufactory and again in series of...,visitors suffering from mental illnesses banne...,no help for mexico kidnapping surge
1,why wont america and nato help us if they wont...,bush puts foot down on georgian conflict,jewish georgian minister thanks to israeli tra...,georgian army flees in disarray as russians ad...,olympic opening ceremony fireworks faked,what were the mossad with fraudulent new zeala...,russia angered by israeli military sale to geo...,an american citizen living in ossetia blames a...,welcome to world war iv now in high definition,georgia move mistake of monumental proportions,...,israel and the us behind the georgian aggression,do not believe tv neither russian nor georgian...,riots are still going on in montreal canada be...,china to overtake us as largest manufacturer,war in south ossetia pics,israeli physicians group condemns state torture,russia has just beaten the united states over ...,perhaps the question about the georgia russia ...,russia is so much better at war,so this is what it come to trading sex for food
2,remember that adorable year old who sang at th...,russia ends georgia operation,if we had no sexual harassment we would have n...,al qa eda is losing support in iraq because of...,ceasefire in georgia putin outmaneuvers the west,why microsoft and intel tried to kill the xo 1...,stratfor the russo georgian war and the balanc...,trying to get sense of this whole georgia russ...,the us military was surprised by the timing an...,beats war drum as iran dumps the dollar,...,troops still in georgia did you know they were...,why russias response to georgia was right,gorbachev accuses of making serious blunder in...,russia georgia and nato cold war two,remember that adorable 62 year old who led you...,war in georgia the israeli connection,all signs point to the us encouraging georgia ...,christopher king argues that the us and nato a...,america the new mexico,bbc news asia pacific extinction by man not cl...
3,refuses israel weapons to attack iran report,when the president ordered to attack tskhinval...,israel clears troops who killed reuters cameraman,britain policy of being tough on drugs is poin...,body of 14 year old found in trunk latest rans...,china has moved 10 million quake survivors int...,bush announces operation get all up in russia ...,russian forces sink georgian ships,the commander of navy air reconnaissance squad...,92 of cnn readers russia actions in georgia ju...,...,elephants extinct by 2020,us humanitarian missions soon in georgia if ru...,georgia ddos came from us sources,russian convoy heads into georgia violating truce,israeli defence minister us against strike on ...,gorbachev we had no choice,witness russian forces head towards tbilisi in...,quarter of russians blame for conflict poll,georgian president says us military will take ...,2006 nobel laureate aleksander solzhenitsyn ac...
4,all the experts admit that we should legalise ...,war in south osetia 89 pictures made by russia...,swedish wrestler ara abrahamian throws away me...,russia exaggerated t

In [6]:
str_tokens_sw.shape

(1989, 25)

In [7]:
def replace_num(element):
    return ' '.join([('numero' if k.isdigit() else k) for k in element.split()])

http://stackoverflow.com/questions/6905636/python-conditional-list-joins

Instead of a digit, use `'NUMERO'` as it is resistant to stemming/lemmatizing.  It's possible that headlines with numbers might contain some information.  This will prevent preprocessing from discarding the information.

In [8]:
str_tokens_sw['merged'] = str_tokens_sw.iloc[:, 0:].apply(lambda x: ' '.join(x.dropna().values.tolist()), axis=1)

In [9]:
# type(str_tokens['merged'][0])

In [10]:
str_tokens_sw['merged'] = str_tokens_sw['merged'].apply(replace_num)

In [11]:
from nltk.stem.porter import PorterStemmer

In [12]:
pstem = PorterStemmer()

In [13]:
def stm(element):
    return ' '.join([pstem.stem(k) for k in element.split()])

In [14]:
str_tokens_sw['merged'] = str_tokens_sw['merged'].apply(stm)

In [15]:
str_tokens_sw['merged']

0       georgia down two russian warplan as countri mo...
1       whi wont america and nato help us if they wont...
2       rememb that ador year old who sang at the open...
3       refus israel weapon to attack iran report when...
4       all the expert admit that we should legalis dr...
5       mom of miss gay man too bad he not numero year...
6       in an afghan prison the major of femal prison ...
7       man arrest and lock up for five hour after tak...
8       two elderli chines women have been sentenc to ...
9       british resid held in guantanamo bay win legal...
10      syria say it readi to put russian missil syste...
11      korea kim die in numero replac by lookalik say...
12      north korea halt denuclearis after us fail to ...
13      photo of numero year old iraqi suicid bomber w...
14      militari help for georgia is declar of war say...
15      russian prime minist vladimir putin ha accus t...
16      girl film isra soldier brutal sinc then her fa...
17      poland

Split labels and tokens into 4 folds
Fold 1 = [:499]
Fold 2 = [499:996]
Fold 3 = [996:1493]
Fold 4 = [1493:]

## Split data into train and test sets

In [16]:
train_text = str_tokens_sw.merged[0:1493] # train features

In [17]:
test_text = str_tokens_sw.merged[1493:] # test features

In [18]:
train_labels = labels[0:1493].values # train labels

In [19]:
test_labels = labels[1493:].values; # test labels

In [20]:
vectorizer = TfidfVectorizer( max_features=250000, ngram_range = ( 1, 4 ), sublinear_tf = True, max_df=0.66 )

In [21]:
# Only need text, not labels
train_x = vectorizer.fit_transform( train_text )

In [22]:
test_x = vectorizer.transform( test_text )

In [23]:
train_x.shape

(1493, 250000)

In [24]:
test_x.shape

(496, 250000)

### Passive Aggressive Classifier

In [25]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [26]:
classifier = PassiveAggressiveClassifier(n_iter=25)

In [27]:
train_labels.shape

(1493, 1)

In [28]:
train_labels.ravel()

array([0, 1, 0, ..., 1, 1, 1])

In [29]:
classifier.fit(train_x, train_labels.ravel())

PassiveAggressiveClassifier(C=1.0, class_weight=None, fit_intercept=True,
              loss='hinge', n_iter=25, n_jobs=1, random_state=None,
              shuffle=True, verbose=0, warm_start=False)

In [30]:
classifier.score(test_x, test_labels.ravel())

0.51209677419354838

### SGD Classifier

In [31]:
from sklearn.linear_model import SGDClassifier

In [32]:
classifier = SGDClassifier(loss='squared_loss', n_iter=25)

In [33]:
classifier.fit(train_x, train_labels.ravel())

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_loss', n_iter=25, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [34]:
classifier.score(test_x, test_labels.ravel())

0.50604838709677424

### Logisitc Regression

In [35]:
from sklearn.linear_model import LogisticRegression

In [36]:
logr = LogisticRegression()

In [37]:
logr.fit(train_x, train_labels.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [38]:
logr.score(test_x, test_labels.ravel())

0.51814516129032262

### Ridge Classifier

In [39]:
from sklearn.linear_model import RidgeClassifier

In [40]:
clf = RidgeClassifier()

In [41]:
clf.fit(train_x, train_labels.ravel())

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [42]:
clf.score(test_x, test_labels.ravel())

0.51814516129032262

### Gassian NB

In [43]:
from sklearn.naive_bayes import GaussianNB

In [44]:
gnb = GaussianNB()

In [45]:
gnb.fit(train_x.toarray(), train_labels.ravel())

GaussianNB(priors=None)

In [46]:
gnb.score(test_x.toarray(), test_labels.ravel())

0.49395161290322581

### Support Vector Classifier

In [47]:
from sklearn.svm import SVC

In [48]:
supportvc = SVC()

In [49]:
supportvc.fit(train_x, train_labels.ravel())

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [50]:
supportvc.score(test_x, test_labels.ravel())

0.5161290322580645